In [ ]:
pip install dicom2nifti

In [ ]:
import dicom2nifti

In [ ]:
pip show dicom2nifti 

In [ ]:
import dicom2nifti
import os
import pydicom  # This is used to test if a file is a valid DICOM file

In [ ]:
# Define the main directory path, here you should put all the patient folders, I used DICOM  
main_dir = r'PATH\TO\YOUR\MAIN_DIRECTORY'

# Output base directory
output_base_dir = r'PATH\TO\YOUR\OUTPUT_DIRECTORY'

# Ensure the output directory exists
if not os.path.exists(output_base_dir):
    os.makedirs(output_base_dir)

In [ ]:
#this will force check the files to see if they are dicom file. I had an issue with some files not having the .dcm extension
def is_dicom_file(filepath):
    try:
        # Attempt to read the file using pydicom
        dcm = pydicom.dcmread(filepath, force=True, stop_before_pixels=True)
        # Check for specific DICOM elements that should be present; adjust according to your needs
        if dcm.file_meta.MediaStorageSOPClassUID:
            return True
    except Exception as e:
        print(f"Failed to read {filepath} as DICOM: {e}")
        return False

In [ ]:
# Iterate over each patient directory in the main directory, with some debug prints for clear understanding
for patient_id in os.listdir(main_dir):
    patient_dir = os.path.join(main_dir, patient_id)

    if os.path.isdir(patient_dir):
        dicom_found = False  # Flag to track presence of DICOM files

        for series_name in os.listdir(patient_dir):
            series_dir = os.path.join(patient_dir, series_name)

            if os.path.isdir(series_dir) and any(is_dicom_file(os.path.join(series_dir, f)) for f in os.listdir(series_dir)):
                dicom_found = True
                try:
                    output_file = os.path.join(output_base_dir, f'{patient_id}.nii.gz')

                    print(f"Starting conversion: {series_dir} to {output_file}")
                    
                    # this is the main conversion. Here I specified "reorient False" for my project, but you can it change here.
                    dicom2nifti.dicom_series_to_nifti(series_dir, output_file, reorient_nifti=False)
                    print(f'Conversion successful for {patient_id}')
                except Exception as e:
                    print(f'Error converting {patient_id}: {str(e)}')

        if not dicom_found:
            print(f"No DICOM files found in any series in patient directory: {patient_dir}")
    else:
        print(f"Not a directory or no access: {patient_dir}")
